In [5]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
from utils import *
from langchain.schema import Document

load_dotenv()

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

# all_texts = load_pdfs_from_folder("data/insurance")
# all_text_chunks = chunk_texts(all_texts)

documents = list()
for file in os.listdir("data/insurance"):
    all_texts = load_text_from_pdf("data/insurance/" + file)
    all_text_chunks = chunk_texts(all_texts)
    documents.extend([
        Document(page_content=chunk, metadata={"source": file, "content": "insurance"}) 
        for chunk in all_text_chunks
    ])


db = FAISS.from_documents(documents, embeddings)
db.save_local("faiss_index") 


In [9]:
from langchain.chat_models import ChatOpenAI

load_dotenv()

# embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
llm = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), model_name="gpt-4o")

db = FAISS.load_local(
    "faiss_index", 
    embeddings, 
    allow_dangerous_deserialization = True
    )

def ask(query):
    docs = db.similarity_search(query, k=3)

    context = "\n\n".join(doc.page_content for doc in docs)
    
    prompt = f"""Answer the question based only on the context below.

Context:
{context}

Question: {query}"""

    response = llm.predict(prompt)

    source_list = set()
    for doc in docs:
        source_list.add(doc.metadata["source"])
    
    source_list = list(source_list)
    source_str = ", ".join(source_list)
    
    response += f"\n\nsources: {source_str}"

    return response

if __name__ == "__main__":
    q = "what is insurance and how does it work?"
    print("\nAnswer:", ask(q))
    q = "What are the main types of insurance?"
    print("\nAnswer:", ask(q))



Answer: Insurance is a contract, represented by a policy, through which a policyholder receives financial protection or reimbursement against losses from an insurance company. The company pools clients' risks to make payments more manageable. Various types of insurance policies exist, such as auto, health, homeowners, and life insurance. Individuals and businesses can find an insurance company willing to provide coverage for a price. The core components of most insurance policies include the premium, deductible, and policy limits.

sources: investopedia_insurance_1.pdf

Answer: The main types of insurance are auto, health, homeowners, and life insurance.

sources: investopedia_insurance_1.pdf, investopedia_insurance_2.pdf
